In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [4]:
# read in latest training set
df_train = pd.read_csv(path/'train_v2.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [5]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
78969,Grant_Hanley,20,2,90,Norwich,Tottenham Hotspur,0.295212,2.001099,True,1,...,2,2,4.0,1237,3686,2449,0,2019-12-28T17:30:00Z,1920,1.0
78970,Roberto_Pereyra,20,3,0,Watford,Aston Villa,0.433797,0.522434,True,0,...,0,3,0.0,83,619,536,0,2019-12-28T15:00:00Z,1920,1.0
78971,Sead_Kolasinac,20,2,0,Arsenal,Chelsea,1.413491,1.757609,True,0,...,2,1,0.0,-1303,6,1309,0,2019-12-29T14:00:00Z,1920,0.0
78972,Stuart_Armstrong,20,3,22,Southampton,Crystal Palace,0.469252,0.464142,True,1,...,1,1,2.0,300,385,85,0,2019-12-28T15:00:00Z,1920,1.0
78973,Rúben Diogo_da Silva Neves,20,3,57,Wolverhampton Wanderers,Liverpool,0.746215,2.460966,False,1,...,0,1,8.0,-4146,2817,6963,0,2019-12-29T16:30:00Z,1920,1.0


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

21

In [6]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [7]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season
0,Ederson_Santana de Moraes,20,1,0,Manchester City,Sheffield United,2.711234,0.242030,True,0,1920
1,Xherdan_Shaqiri,20,3,0,Liverpool,Wolverhampton Wanderers,2.460966,0.746215,True,0,1920
2,Hugo_Lloris,20,1,0,Tottenham Hotspur,Norwich,2.001099,0.295212,False,0,1920
3,Cenk_Tosun,20,4,0,Everton,Newcastle United,1.033814,0.561705,False,0,1920
4,Callum_Robinson,20,3,58,Sheffield United,Manchester City,0.242030,2.711234,False,1,1920
5,Heung-Min_Son,20,3,0,Tottenham Hotspur,Norwich,2.001099,0.295212,False,0,1920
6,João Filipe Iria_Santos Moutinho,20,3,90,Wolverhampton Wanderers,Liverpool,0.746215,2.460966,False,2,1920
7,Andrew_Robertson,20,2,90,Liverpool,Wolverhampton Wanderers,2.460966,0.746215,True,6,1920
8,Danny_Rose,20,2,0,Tottenham Hotspur,Norwich,2.001099,0.295212,False,0,1920
9,Martin_Kelly,20,2,90,Crystal Palace,Southampton,0.464142,0.469252,False,2,1920


In [8]:
df_latest.shape

(584, 11)

In [9]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [10]:
last_play_proba.head()

,player,play_proba
31,Shkodran_Mustafi,1.00
32,Héctor_Bellerín,0.75
33,Sead_Kolasinac,0.00
34,Ainsley_Maitland-Niles,1.00
35,Sokratis_Papastathopoulos,0.75


In [11]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [12]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
0,Ederson_Santana de Moraes,20,1,0,Manchester City,Sheffield United,2.711234,0.242030,True,0,1920,0.00
1,Xherdan_Shaqiri,20,3,0,Liverpool,Wolverhampton Wanderers,2.460966,0.746215,True,0,1920,1.00
2,Hugo_Lloris,20,1,0,Tottenham Hotspur,Norwich,2.001099,0.295212,False,0,1920,0.00
3,Cenk_Tosun,20,4,0,Everton,Newcastle United,1.033814,0.561705,False,0,1920,1.00
4,Callum_Robinson,20,3,58,Sheffield United,Manchester City,0.242030,2.711234,False,1,1920,1.00
5,Heung-Min_Son,20,3,0,Tottenham Hotspur,Norwich,2.001099,0.295212,False,0,1920,0.00
6,João Filipe Iria_Santos Moutinho,20,3,90,Wolverhampton Wanderers,Liverpool,0.746215,2.460966,False,2,1920,1.00
7,Andrew_Robertson,20,2,90,Liverpool,Wolverhampton Wanderers,2.460966,0.746215,True,6,1920,1.00
8,Danny_Rose,20,2,0,Tottenham Hotspur,Norwich,2.001099,0.295212,False,0,1920,1.00
9,Martin_Kelly,20,2,90,Crystal Palace,Southampton,0.464142,0.469252,False,2,1920,1.00


In [13]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [14]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
78969,Grant_Hanley,20,2,90,Norwich,Tottenham Hotspur,0.295212,2.001099,True,1,1920,1.0
78970,Roberto_Pereyra,20,3,0,Watford,Aston Villa,0.433797,0.522434,True,0,1920,1.0
78971,Sead_Kolasinac,20,2,0,Arsenal,Chelsea,1.413491,1.757609,True,0,1920,0.0
78972,Stuart_Armstrong,20,3,22,Southampton,Crystal Palace,0.469252,0.464142,True,1,1920,1.0
78973,Rúben Diogo_da Silva Neves,20,3,57,Wolverhampton Wanderers,Liverpool,0.746215,2.460966,False,1,1920,1.0


In [15]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [16]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [17]:
fixtures.head(10)

,home_team,away_team,gw
179,West Ham United,Liverpool,24
200,Arsenal,Manchester United,21
201,Brighton and Hove Albion,Chelsea,21
202,Burnley,Aston Villa,21
203,Liverpool,Sheffield United,21
204,Manchester City,Everton,21
205,Newcastle United,Leicester City,21
206,Norwich,Crystal Palace,21
207,Southampton,Tottenham Hotspur,21
208,Watford,Wolverhampton Wanderers,21


In [18]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [19]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,24,West Ham United,Liverpool,21,14
1,21,Arsenal,Manchester United,3,1
2,21,Brighton and Hove Albion,Chelsea,36,8
3,21,Burnley,Aston Villa,90,7
4,21,Liverpool,Sheffield United,14,49


In [20]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [21]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [22]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,24,Liverpool,West Ham United,Andrew_Robertson,2,70,100,False
1,24,Liverpool,West Ham United,Trent_Alexander-Arnold,2,74,100,False
2,24,Liverpool,West Ham United,Virgil_van Dijk,2,64,100,False
3,24,Liverpool,West Ham United,Joseph_Gomez,2,51,100,False
4,24,Liverpool,West Ham United,Joel_Matip,2,52,0,False


In [23]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,24,West Ham United,Liverpool,Aaron_Cresswell,2,47,100,True
1,24,West Ham United,Liverpool,Angelo_Ogbonna,2,45,None,True
2,24,West Ham United,Liverpool,Arthur_Masuaku,2,43,100,True
3,24,West Ham United,Liverpool,Pablo_Zabaleta,2,42,None,True
4,24,West Ham United,Liverpool,Ryan_Fredericks,2,45,100,True


In [24]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 22.333333333333332,
 'Aaron_Cresswell': 70.0,
 'Aaron_Lennon': 11.0,
 'Aaron_Mooy': 88.6,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 72.0,
 'Abdoulaye_Doucouré': 80.25,
 'Adam_Idah': 0.0,
 'Adam_Lallana': 24.666666666666668,
 'Adam_Masina': 23.75,
 'Adam_Webster': 72.0,
 'Adama_Traoré': 78.4,
 'Addji Keaninkin Marc-Israel_Guehi': 0.0,
 'Adrian_Mariappa': 65.2,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 54.0,
 'Ainsley_Maitland-Niles': 90.0,
 'Akin_Famewo': 0.0,
 'Albian_Ajeti': 5.25,
 'Alex_Iwobi': 63.333333333333336,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 53.0,
 'Alexander_Tettey': 85.0,
 'Alexandre_Lacazette': 38.4,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 21.0,
 'Alisson_Ramses Becker': 90.0,
 'Andre_Gray': 7.6,
 'Andreas_Christensen': 18.0,
 'Andreas_Pereira': 35.6,
 'Andrew_Robertson': 71.5,
 'Andrew_Surman': 0.0,
 'Andriy_Yarmolenko': 0.0,
 'Andros_Townsend': 36.0,
 'Andy_Carroll': 60.2,
 'Andy_Lonergan': 0.0,
 'Angel_

In [25]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [26]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,24,West Ham United,Liverpool,Aaron_Cresswell,2,4.7,1.00,True,0.699185,2.460966,1920,70.000000
1,24,West Ham United,Liverpool,Angelo_Ogbonna,2,4.5,1.00,True,0.699185,2.460966,1920,67.500000
2,24,West Ham United,Liverpool,Arthur_Masuaku,2,4.3,1.00,True,0.699185,2.460966,1920,32.250000
3,24,West Ham United,Liverpool,Pablo_Zabaleta,2,4.2,1.00,True,0.699185,2.460966,1920,22.500000
4,24,West Ham United,Liverpool,Ryan_Fredericks,2,4.5,1.00,True,0.699185,2.460966,1920,90.000000
5,24,West Ham United,Liverpool,Issa_Diop,2,4.4,1.00,True,0.699185,2.460966,1920,23.000000
6,24,West Ham United,Liverpool,Fabián_Balbuena,2,4.3,1.00,True,0.699185,2.460966,1920,90.000000
7,24,West Ham United,Liverpool,Winston_Reid,2,3.9,1.00,True,0.699185,2.460966,1920,0.000000
8,24,West Ham United,Liverpool,Ben_Johnson,2,4.0,1.00,True,0.699185,2.460966,1920,0.000000
9,24,West Ham United,Liverpool,Marko_Arnautovic,4,7.0,0.00,True,0.699185,2.460966,1920,0.000000


In [27]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')